In [1]:
import sympy as sp
import numpy as np
import itertools


# define symbols
q = sp.symbols("q")
t = sp.symbols("t")
I2 = sp.Matrix(np.identity(2))

In [2]:
def mt (a, b):
    """
    Calculate the tensor product of two matrices
    Ordering: (1, 2) < (2, 1), (0, 1) < (1, 0)
    
    Parameters:
    a (array-like): M by N matrix
    b (array-like): P by Q matrix
    
    Returns:
    array: MP by NQ matrix
    """
 
    M, N = a.shape
    P, Q = b.shape
    t = sp.Matrix(np.zeros((M * P, N * Q)))

    for i1, i2, j1, j2 in itertools.product(range(M), range(N), range(P), range(Q)):
        t[i1 * P + j1, i2 * Q + j2] = a[i1, i2] * b[j1, j2]

    return t

def mt_many (*matrices):
    """
    Calculate the tensor product finitely many martices using mt function
    Ordering: (1, 2) < (2, 1), (0, 1) < (1, 0)
    """
    m0 = matrices[0]
    for m in matrices[1:]:
        m0 = mt(m0, m)
    return m0

In [29]:
EV1 = sp.Matrix([[0, 1], [0, 0]])
FV1 = sp.Matrix([[0, 0], [1, 0]])
KV1 = sp.Matrix([[q, 0], [0, q**(-1)]])
Kinv = sp.Matrix([[q**(-1), 0], [0, q]])

# Apply coproduct to E, F, K, Kinv
DE = mt(I2, EV1) + mt(EV1, KV1)
DF = mt(Kinv, FV1) + mt(FV1, I2)
DK = mt(KV1, KV1)
DKinv = mt(Kinv, Kinv)

# Apply antipode S to E, F, K, Kinv
SE = - EV1 * Kinv 
SF = - KV1 * FV1
SK = Kinv
SKinv = KV1

---

$R$ matrix on $V \otimes V$


``sp.Matrix([[q**(1/2), 0, 0, 0], [0, q**(-1/2), 0, 0], [0, 0, q**(-1/2), 0], [0, 0,  0, q**(1/2)]]) * \
    (sp.Matrix(np.identity(4)) + (q-q**(-1)) * mt(EV1, FV1))``


$q^{-1/2}\begin{bmatrix}
q & 0 & 0 & 0 \\
0 & 1 & q-q^{-1} & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & q
\end{bmatrix}$

For some unknown reason, if we scale $C_{V,V}=\tau_{V,V}\circ R_{V,V}$ and $C_{V*,V*}=\tau_{V*,V*}\circ R_{V*,V*}$ by a scalar $q^3$, then we can still get the Skein relation, while making the invariant independent to choices of decomposition. 

In [31]:
RVV = sp.Matrix([[q**(1/2), 0, 0, 0], [0, q**(-1/2), 0, 0], [0, 0, q**(-1/2), 0], [0, 0,  0, q**(1/2)]]) * \
    (sp.Matrix(np.identity(4)) + (q-q**(-1)) * mt(EV1, FV1))
RVVinv = RVV.inv()


#Check that tauAA \circ \Delta(x) = RV \circ \Delta(x) \circ RV^{-1} is satisfied for any x \in A=Uq(sl2)
#print(RVV * DE * RVVinv)
#print(mt(EV1, I2) + mt(KV1, EV1))
#print(RVV * DF * RVVinv)
#print(mt(FV1, Kinv) + mt(I2, FV1))
#print(RVV * DK * RVVinv)
#print(mt(KV1, KV1))

RVV_ = sp.Matrix([[q**(-1/2), 0, 0, 0], [0, q**(1/2), 0, 0], [0, 0, q**(1/2), 0], [0, 0,  0, q**(-1/2)]]) * \
       (sp.Matrix(np.identity(4)) + (q-q**(-1)) * mt(EV1, SF.transpose()))
RVV_inv = RVV_.inv()
#RVV_inv = sp.Matrix([[q**(1/2), 0, 0, 0], [0, q**(-1/2), 0, 0], [0, 0, q**(-1/2), 0], [0, 0,  0, q**(1/2)]]) * \
#          (sp.Matrix(np.identity(4)) + (-q+q**(-1)) * mt(EV1, SF.transpose())) * q**(5/2)

#print(RVV_ * (mt(I2, SE.transpose()) + mt(EV1, SK.transpose())) * RVV_inv) # R \Delta(E) R^{-1}
#print(mt(SE.transpose(), I2) + mt(SK.transpose(), EV1))                   # tauAA \Delta(E) This is wrong
#print(mt(EV1, I2) + mt(KV1, SE.transpose()))                               # This is the correct tauAA \Delta(E)

RV_V = sp.Matrix([[q**(-1/2), 0, 0, 0], [0, q**(1/2), 0, 0], [0, 0, q**(1/2), 0], [0, 0,  0, q**(-1/2)]]) * \
       (sp.Matrix(np.identity(4)) + (q-q**(-1)) * mt(SE.transpose(), FV1)) 
RV_Vinv = RV_V.inv()
#RV_Vinv = sp.Matrix([[q**(1/2), 0, 0, 0], [0, q**(-1/2), 0, 0], [0, 0, q**(-1/2), 0], [0, 0,  0, q**(1/2)]]) * \
#          (sp.Matrix(np.identity(4)) + (-q+q**(-1)) * mt(SE.transpose(), FV1))

RV_V_ = sp.Matrix([[q**(1/2), 0, 0, 0], [0, q**(-1/2), 0, 0], [0, 0, q**(-1/2), 0], [0, 0,  0, q**(1/2)]]) * \
    (sp.Matrix(np.identity(4)) + (q-q**(-1)) * mt(SE.transpose(), SF.transpose())) 
RV_V_inv = RV_V_.inv()


RVV

Matrix([
[1.0*q**0.5,          0,                0,          0],
[         0, 1.0/q**0.5, (q - 1/q)/q**0.5,          0],
[         0,          0,       1.0/q**0.5,          0],
[         0,          0,                0, 1.0*q**0.5]])

### Note
Checked that all of $R_{V,V},\,R_{V,V*},\,R_{V*,V}$ satisfy the $R$ matrix axiom $R \Delta(x) R^{-1} = \tau_{A,A}( \Delta(x))$

---

THETA has to act by a constant on $V$. By the axiom $(\theta_V)^*=\theta_{V^*}$ it has to act by the same scalar on $V^*$.

In [34]:
# CVV = q**(2) * sp.Matrix([[q**(1), 0, 0, 0], [0, 0, 1, 0], [0, 1, q**(1)-q**(-1), 0], [0, 0, 0, q**(1)]])

tauVV = sp.Matrix([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
CVV = tauVV * RVV * q**(-3)
CVVinv = CVV.inv()
CVV_ = tauVV * RVV_
CVV_inv = CVV_.inv()
CV_V = tauVV * RV_V
CV_Vinv = CV_V.inv()
CV_V_ = tauVV * RV_V_ * q**(-3)
CV_V_inv = CV_V_.inv()

E = sp.Matrix([[1, 0, 0, 1]])
I = sp.Matrix([[1], [0], [0], [1]])



THETA = q**(3/2)
# fac = q**(5/2)
# THETA = THETA0 * fac**(-1)



I1 = THETA * CVV_ * I 
E1 = THETA * E  * CVV_ 


THETA

q**1.5

For invariants do not modify $\theta^{w(L)}$ but notice the new value of $\theta$

$w(L)$ is the number of not-inversed C matrices - number of inversed C matrices

In [35]:
pol = E * I1 # check unknot # get q+q**(-1)
sp.simplify(sp.nsimplify(pol, rational=True))

Matrix([[q + 1/q]])

In [36]:
pol = E1 * I # check unknot # get q+q**(-1)
sp.simplify(sp.nsimplify(pol, rational=True))

Matrix([[q + 1/q]])

In [37]:
mat = mt(E1, np.identity(2)) * mt(np.identity(2), I1) # Verify dual snake 1 # Good 
sp.simplify(sp.nsimplify(mat, rational=True))

Matrix([
[1, 0],
[0, 1]])

In [38]:
mat = mt(np.identity(2), E1) * mt(I1, np.identity(2)) # Verify dual snake 2 # Good 
sp.simplify(sp.nsimplify(mat, rational=True))

Matrix([
[1, 0],
[0, 1]])

In [39]:
mat = THETA * mt(E, I2) * mt(CVV_, I2) * mt(I2, I1) # Fig 3.10 expecting 1, getting 1 Good
sp.simplify(sp.nsimplify(mat, rational=True))

Matrix([
[1, 0],
[0, 1]])

In [40]:
mat = THETA**(-1) * mt(I2, E1) * mt(I2, CVV_inv) * mt(I, I2) # expecting 1 getting 1 Good
sp.simplify(sp.nsimplify(mat, rational=True))

Matrix([
[1, 0],
[0, 1]])

In [41]:
mat = THETA**(-1) * mt(E, I2) * mt(CV_Vinv, I2) * mt(I2, I1) # R1 expecting 1 getting 1 
sp.simplify(sp.nsimplify(mat, rational=True))

Matrix([
[1, 0],
[0, 1]])

In [42]:
# Note the negative crossing is consistant
mat = THETA**(-1) * mt(E1, I2) * mt(CVV_inv, I2) * mt(I2, I) # R1 change V V* labelling expecting 1 getting 1 
sp.simplify(sp.nsimplify(mat, rational=True))

Matrix([
[1, 0],
[0, 1]])

In [43]:
# Another decomposition of R1, expecting 1, getting q**3
mat = THETA**(1) * mt(E, I2) * mt(I2, CVV) * mt(I1, I2)
sp.simplify(sp.nsimplify(mat, rational=True))
# But if using the new way of defining positive/negative crossings it should have a negative crossing
#mat = THETA**(1) * mt(E1, I2) * mt(I2, CV_V_*q**(-3)) * mt(I, I2)
#sp.simplify(sp.nsimplify(mat, rational=True))



Matrix([
[1, 0],
[0, 1]])

In [44]:
# unknot also has q**3 factor
mat = THETA**(1) * mt_many(E1,  E) * mt_many(I2, CV_V_, I2) * mt_many(I, I1)
sp.simplify(sp.nsimplify(mat, rational=True))

Matrix([[q + 1/q]])

In [47]:
# R3 V V* V expecting 0 getting 0
diffR3 = mt(CVV_inv, I2) * mt(I2, CVVinv) * mt(CVV_, I2) - mt(I2, CVV_) * mt(CVVinv, I2) * mt(I2, CVV_inv)
sp.simplify(sp.nsimplify(diffR3, rational=True))


# R3 V* V V* expecting 0 getting 0
diffR3 = mt(CV_Vinv, I2) * mt(I2, CV_V_inv) * mt(CV_V, I2) - mt(I2, CV_V) * mt(CV_V_inv, I2) * mt(I2, CV_Vinv)
sp.simplify(sp.nsimplify(diffR3, rational=True))

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0]])

In [48]:
expr = mt_many(E1, E) * mt_many(CVV_inv, I2, I2)  * mt_many(I2, I, I2) * mt_many(I2, E, I2) * mt_many(I2, CVV_, I2) * mt_many(I1, I1) 
sp.simplify(sp.nsimplify(expr[0], rational=True))
# expecting unknot invariant, good

q + 1/q

In [49]:
# Trefoil A, right handed
invt_A = mt_many(E, E) * mt_many(I2, I, I2) * mt_many(I2, E, I2) * mt_many(I2, I2, CVV) * \
         mt_many(CV_V_, I2, I2) * mt_many(I2, CV_Vinv, I2) * mt_many(I1, I1) * THETA / \
         (q + q**(-1)) 
invt_A_raional = sp.cancel(sp.simplify(sp.nsimplify(invt_A[0], rational=True)))
#sp.factor(invt_A[0])
#num, deno = invt_A_raional.as_numer_denom()
poly = sp.simplify(invt_A_raional)
print(poly)


coef_expo_A = {}
for term in poly.as_ordered_terms():
    coef, expo = term.as_coeff_exponent(q)
    coef_expo_A[expo] = coef
    print(f"coefficient={coef}, exponent={expo}")

leading, trailing = max(coef_expo_A.keys()), min(coef_expo_A.keys())
norm_neg =  {key-leading: value for key, value in coef_expo_A.items()}
norm_pos =  {key-trailing: value for key, value in coef_expo_A.items()}

norm_neg_poly = sum(coef * q**expo for expo, coef in norm_neg.items())

norm_neg_poly = sum(norm_neg[expo] * t**(expo/2) for expo in sorted(norm_neg.keys(), reverse=True))
#norm_pos_poly = sum(coef * x**expo for expo, coef in norm_pos.items())
norm_pos_poly = sum(norm_pos[expo] * t**(expo/2) for expo in sorted(norm_pos.keys(), reverse=True))
print("Normalized polynomial (leading term 1):", norm_neg_poly)
print("Normalized polynomial (trailing term 1):", norm_pos_poly)

(q**6 + q**2 - 1)/q**8
coefficient=(q**6 + q**2 - 1)/q**8, exponent=0
Normalized polynomial (leading term 1): (q**6 + q**2 - 1)/q**8
Normalized polynomial (trailing term 1): (q**6 + q**2 - 1)/q**8


In [52]:
# Trefoil A ver 2 
invt_A = mt_many(E, E) * mt_many(I2, I, I2) * mt_many(I2, E, I2) * mt_many(I2, I2, CVV) * \
         mt_many(CV_V_, I2, I2) * mt_many(I2, I2, CVV) * mt_many(I2, I1, I2) * mt_many(I2, E1, I2) * mt_many(I1, I1) * \
         THETA**3 / (q + q**(-1))

invt_A_raional = sp.cancel(sp.simplify(sp.nsimplify(invt_A[0], rational=True)))
#sp.factor(invt_A[0])
#num, deno = invt_A_raional.as_numer_denom()
invt_A_raional = sp.nsimplify(invt_A_raional, rational=True)
print(invt_A_raional)


num, deno = invt_A_raional.as_numer_denom()
poly = 0
for terms in num.as_ordered_terms():
    poly = poly + terms / deno
poly = sp.nsimplify(poly, rational=True)
print(poly)


coef_expo_A = {}
for term in poly.as_ordered_terms():
    coef, expo = term.as_coeff_exponent(q)
    coef_expo_A[expo] = coef
    print(f"coefficient={coef}, exponent={expo}")

leading, trailing = max(coef_expo_A.keys()), min(coef_expo_A.keys())
norm_neg =  {key-leading: value for key, value in coef_expo_A.items()}
norm_pos =  {key-trailing: value for key, value in coef_expo_A.items()}

norm_neg_poly = sum(norm_neg[expo] * t**(expo/2) for expo in sorted(norm_neg.keys(), reverse=True))
#norm_pos_poly = sum(coef * x**expo for expo, coef in norm_pos.items())
norm_pos_poly = sum(norm_pos[expo] * t**(expo/2) for expo in sorted(norm_pos.keys(), reverse=True))
print("Normalized polynomial (leading term 1):", norm_neg_poly)
print("Normalized polynomial (trailing term 1):", norm_pos_poly)

(q**6 + q**2 - 1)/q**8
q**(-2) + q**(-6) - 1/q**8
coefficient=1, exponent=-2
coefficient=1, exponent=-6
coefficient=-1, exponent=-8
Normalized polynomial (leading term 1): 1 + t**(-2) - 1/t**3
Normalized polynomial (trailing term 1): t**3 + t - 1


In [53]:
# Trefoil B (1)
invt_B = mt_many(E, E) * mt_many(I2, CV_V, I2) * mt_many(CV_V_inv, I2, I2) * mt_many(I2, CVV_, I2) * \
         mt_many(I1, I1) * THETA / (q + q**(-1))

invt_B_raional = sp.cancel(sp.simplify(sp.nsimplify(invt_B[0], rational=True)))
#poly = sp.nsimplify(invt_B_raional, rational=True)
num, deno = invt_B_raional.as_numer_denom()
poly = 0
for terms in num.as_ordered_terms():
    poly = poly + terms / deno
poly = sp.nsimplify(poly, rational=True)
print(poly)


coef_expo_B = {}
for term in poly.as_ordered_terms():
    coef, expo = term.as_coeff_exponent(q)
    coef_expo_B[expo] = coef
    print(f"coefficient={coef}, exponent={expo}")

print("Jones polynomial is", sum(coef_expo_B[expo] * t**(expo/2) for expo in coef_expo_B.keys()))

leading, trailing = max(coef_expo_B.keys()), min(coef_expo_B.keys())
norm_neg =  {key-leading: value for key, value in coef_expo_B.items()}
norm_pos =  {key-trailing: value for key, value in coef_expo_B.items()}


norm_neg_poly = sum(norm_neg[expo] * t**(expo/2) for expo in sorted(norm_neg.keys(), reverse=True))
#norm_pos_poly = sum(coef * x**expo for expo, coef in norm_pos.items())
norm_pos_poly = sum(norm_pos[expo] * t**(expo/2) for expo in sorted(norm_pos.keys(), reverse=True))
print("Normalized polynomial (leading term 1):", norm_neg_poly)
print("Normalized polynomial (trailing term 1):", norm_pos_poly)

-q**8 + q**6 + q**2
coefficient=-1, exponent=8
coefficient=1, exponent=6
coefficient=1, exponent=2
Jones polynomial is -t**4 + t**3 + t
Normalized polynomial (leading term 1): -1 + 1/t + t**(-3)
Normalized polynomial (trailing term 1): -t**3 + t**2 + 1


In [54]:
# Trefoil B (2)
invt_B2 = mt_many(E, E) * mt_many(I2, I, I2) * mt_many(I2, E, I2) * mt_many(I2, I2, CVVinv) * \
         mt_many(CV_V_inv, I2, I2) * mt_many(I2, CVV_, I2) * mt_many(I1, I1) * THETA**(-1) / (q + q**(-1))
sp.simplify(invt_B2)

invt_B_raional = sp.cancel(sp.simplify(sp.nsimplify(invt_B2[0], rational=True)))
#poly = sp.nsimplify(invt_B_raional, rational=True)
num, deno = invt_B_raional.as_numer_denom()
poly = 0
for terms in num.as_ordered_terms():
    poly = poly + terms / deno
poly = sp.nsimplify(poly, rational=True)
print(poly)


coef_expo_B = {}
for term in poly.as_ordered_terms():
    coef, expo = term.as_coeff_exponent(q)
    coef_expo_B[expo] = coef
    print(f"coefficient={coef}, exponent={expo}")

leading, trailing = max(coef_expo_B.keys()), min(coef_expo_B.keys())
norm_neg =  {key-leading: value for key, value in coef_expo_B.items()}
norm_pos =  {key-trailing: value for key, value in coef_expo_B.items()}

norm_neg_poly = sum(coef * q**expo for expo, coef in norm_neg.items())

norm_neg_poly = sum(norm_neg[expo] * t**(expo/2) for expo in sorted(norm_neg.keys(), reverse=True))
#norm_pos_poly = sum(coef * x**expo for expo, coef in norm_pos.items())
norm_pos_poly = sum(norm_pos[expo] * t**(expo/2) for expo in sorted(norm_pos.keys(), reverse=True))
print("Normalized polynomial (leading term 1):", norm_neg_poly)
print("Normalized polynomial (trailing term 1):", norm_pos_poly)

-q**8 + q**6 + q**2
coefficient=-1, exponent=8
coefficient=1, exponent=6
coefficient=1, exponent=2
Normalized polynomial (leading term 1): -1 + 1/t + t**(-3)
Normalized polynomial (trailing term 1): -t**3 + t**2 + 1


In [55]:
# Trefoil B (3)
invt_B2 = mt_many(E, E) * mt_many(I2, CV_V, I2) * mt_many(CV_V_inv, CVVinv) * mt_many(I2, I1, I2) * \
         mt_many(I2, E1, I2) * mt_many(I1, I1) * THETA**(-1) / (q + q**(-1))
sp.simplify(invt_B2)

invt_B_raional = sp.cancel(sp.simplify(sp.nsimplify(invt_B2[0], rational=True)))
#poly = sp.nsimplify(invt_B_raional, rational=True)
num, deno = invt_B_raional.as_numer_denom()
poly = 0
for terms in num.as_ordered_terms():
    poly = poly + terms / deno
poly = sp.nsimplify(poly, rational=True)
print(poly)


coef_expo_B = {}
for term in poly.as_ordered_terms():
    coef, expo = term.as_coeff_exponent(q)
    coef_expo_B[expo] = coef
    print(f"coefficient={coef}, exponent={expo}")

leading, trailing = max(coef_expo_B.keys()), min(coef_expo_B.keys())
norm_neg =  {key-leading: value for key, value in coef_expo_B.items()}
norm_pos =  {key-trailing: value for key, value in coef_expo_B.items()}

norm_neg_poly = sum(coef * q**expo for expo, coef in norm_neg.items())

norm_neg_poly = sum(norm_neg[expo] * t**(expo/2) for expo in sorted(norm_neg.keys(), reverse=True))
#norm_pos_poly = sum(coef * x**expo for expo, coef in norm_pos.items())
norm_pos_poly = sum(norm_pos[expo] * t**(expo/2) for expo in sorted(norm_pos.keys(), reverse=True))
print("Normalized polynomial (leading term 1):", norm_neg_poly)
print("Normalized polynomial (trailing term 1):", norm_pos_poly)

-q**8 + q**6 + q**2
coefficient=-1, exponent=8
coefficient=1, exponent=6
coefficient=1, exponent=2
Normalized polynomial (leading term 1): -1 + 1/t + t**(-3)
Normalized polynomial (trailing term 1): -t**3 + t**2 + 1


In [56]:
expr = mt(I2, E) * mt(CV_V_inv, np.identity(2)) * mt(I2, I1) * THETA**(-1)
sp.simplify(sp.nsimplify(expr, rational=True))

Matrix([
[1, 0],
[0, 1]])

In [57]:
skein = q**(2) *  THETA**(1) * CVV - q**(-2) *  THETA**(-1) * CVVinv
sp.simplify(sp.nsimplify(skein, rational=True))

Matrix([
[q - 1/q,       0,       0,       0],
[      0, q - 1/q,       0,       0],
[      0,       0, q - 1/q,       0],
[      0,       0,       0, q - 1/q]])

$q^{-2} P_{L+}(q) - q^{2} P_{L-}(q) = q-q^{-1}$

Substitute variable $t=q^{2}$ to get the standard Skein relation

$t^{-1} P_{L+}(q) - t^{1} P_{L-}(q) = t^{1/2}-t^{-1/2}$



In [58]:
#knot 6_3 
prod = mt_many(E, E) * \
       mt_many(I2, I, I2, E) * \
       mt_many(I2, E1, CVV_inv, I2) * \
       mt_many(CVV_, CV_V_, I2, I2) * \
       mt_many(I2, CV_V_inv, CV_Vinv, I2) * \
       mt_many(CV_V, I2, I2, I1) * \
       mt_many(I2, I, I2) * \
       mt_many(I2, E1, I2) * \
       mt_many(I1, I1) *\
       THETA**(0) / (q + q**(-1))  # writhe number = 0, divided by unknot invariant q + q**(-1)

# get and simplify the (1,1) entry of the resulting 1 by 1 matrix
# six_3 is a rational function in q that can be simplified to a polynomial
six_3 = sp.cancel(sp.simplify(sp.nsimplify(prod[0], rational=True))) 
# get the numerator and denominator of the rational function
num, deno = six_3.as_numer_denom()
poly = 0
for terms in num.as_ordered_terms():
    poly = poly + terms / deno
poly = sp.nsimplify(poly, rational=True)
print(poly)

coef_expo = {} # create a dictionary for q polynomial, key: exponent, value: coefficient
for term in poly.as_ordered_terms():
    coef, expo = term.as_coeff_exponent(q)
    coef_expo[expo] = coef 

print(coef_expo)

# Substitute q^2=t and sort the polynomial by exponent
sum(coef_expo[expo] * t**(expo/2) for expo in coef_expo.keys())
    


-q**6 + 2*q**4 - 2*q**2 + 3 - 2/q**2 + 2/q**4 - 1/q**6
{6: -1, 4: 2, 2: -2, 0: 3, -2: -2, -4: 2, -6: -1}


-t**3 + 2*t**2 - 2*t + 3 - 2/t + 2/t**2 - 1/t**3